In [14]:
import numpy as np 
import pandas as pd 
# Import Statsmodels
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic
import pandas as pd
import matplotlib.pyplot as plt

# load predict date
pred_date = pd.read_csv("to_predict.csv")
pred_date=pred_date[pred_date.station_no==6335050].drop('station_no',axis=1)
pred_date.date = pd.to_datetime(pred_date.date)
pred_date

,date
0,1965-04-17
1,1965-04-18
2,1965-04-19
3,1965-08-01
4,1965-08-02
...,...
490,2012-08-07
491,2012-08-08
492,2012-11-20
493,2012-11-21


In [38]:
weather = pd.read_csv('clean_weather.csv')
weather.index = pd.to_datetime(weather.MESS_DATUM)
weather = weather.drop(['SH_TAG','NSH_TAG','MESS_DATUM'],axis=1)
weather

,RS,RSF
MESS_DATUM,,
1965-01-01,1.2,1.0
1965-01-02,0.3,1.0
1965-01-03,0.1,1.0
1965-01-04,0.0,7.0
1965-01-05,1.2,1.0
...,...,...
2012-12-27,10.7,4.0
2012-12-28,1.3,4.0
2012-12-29,0.1,4.0


In [66]:
weather.isna().sum()

RS     0
RSF    0
dtype: int64

In [60]:
weather = weather.replace(-999,np.nan)
weather['RSF'] = weather['RSF'].interpolate(method='nearest')
weather = weather.fillna(weather['RS'].mean())

weather

,RS,RSF
MESS_DATUM,,
1965-01-01,1.2,1.0
1965-01-02,0.3,1.0
1965-01-03,0.1,1.0
1965-01-04,0.0,7.0
1965-01-05,1.2,1.0
...,...,...
2012-12-27,10.7,4.0
2012-12-28,1.3,4.0
2012-12-29,0.1,4.0


In [72]:
# load river data
data = pd.read_csv("river_data.csv", index_col=False) 

df = data[data.station_no.isin([6335020,6335050,6335045,6335070,6335060])]

# df.index = pd.to_datetime(station_1.date)
# df = station_1.drop("date",axis=1)
df = df[df.water_level!=-999]

stations = [6335020,6335050,6335045,6335060]
df_5020 = df[df.station_no==stations[0]]
df_5020.index = pd.to_datetime(df_5020.date)
del df_5020['date']
df_5050 = df[df.station_no==stations[1]]
df_5050.index = pd.to_datetime(df_5050.date)
del df_5050['date']
df_5045 = df[df.station_no==stations[2]]
df_5045.index = pd.to_datetime(df_5045.date)
del df_5045['date']
df_5060 = df[df.station_no==stations[3]]
df_5060.index = pd.to_datetime(df_5060.date)
del df_5060['date']
# add other stations' water level as features
df_5050['w_5020'] = df_5020.water_level
df_5050['w_5060'] =  df_5060.water_level
df_5050['w_5045'] =  df_5045.water_level

df_5050['RS'] =weather.RS
# df_5050['RSF'] = weather.RSF
# df_5050['RSF'] =df_5050['RSF'].interpolate(method='nearest')
df_5050 = df_5050.fillna(weather['RS'].mean())

df_5050 = df_5050.drop("station_no",axis=1)
df_5050 = df_5050.rename(columns = {'discharge':'d_5050', 'water_level':'w_5050'})
df_5050

C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stab

,d_5050,w_5050,w_5020,w_5060,w_5045,RS,RSF
date,,,,,,,
1965-01-01,1320.00,219.0,163.0,110.0,168.93,1.2,1.0
1965-01-02,1650.00,272.0,210.0,178.0,182.48,0.3,1.0
1965-01-03,1950.00,315.0,260.0,200.0,169.97,0.1,1.0
1965-01-04,1870.00,304.0,270.0,195.0,150.31,0.0,7.0
1965-01-05,1870.00,304.0,265.0,196.0,133.09,1.2,1.0
...,...,...,...,...,...,...,...
2012-12-27,5988.73,701.0,808.0,739.0,244.61,10.7,4.0
2012-12-28,5991.16,701.0,804.0,751.0,246.22,1.3,4.0
2012-12-29,6345.64,731.0,811.0,780.0,217.47,0.1,4.0


In [73]:
first = df_5050.copy()
first.isna().sum()

d_5050    0
w_5050    0
w_5020    0
w_5060    0
w_5045    0
RS        0
RSF       0
dtype: int64

## For 6335050

In [74]:
from statsmodels.tsa.stattools import grangercausalitytests
maxlag=12
test = 'ssr_chi2test'
def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    
    """Check Granger Causality of all possible combinations of the Time series.
    The rows are the response variable, columns are predictors. The values in the table 
    are the P-Values. P-Values lesser than the significance level (0.05), implies 
    the Null Hypothesis that the coefficients of the corresponding past values is 
    zero, that is, the X does not cause Y can be rejected.

    data      : pandas dataframe containing the time series variables
    variables : list containing names of the time series variables.
    """
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df

grangers_causation_matrix(first,variables = first.columns)

C:\Users\pc\Anaconda3\lib\site-packages\statsmodels\base\model.py:1532: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 2, but rank is 1
  'rank is %d' % (J, J_), ValueWarning)
C:\Users\pc\Anaconda3\lib\site-packages\statsmodels\base\model.py:1532: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 3, but rank is 2
  'rank is %d' % (J, J_), ValueWarning)
C:\Users\pc\Anaconda3\lib\site-packages\statsmodels\base\model.py:1532: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 4, but rank is 1
  'rank is %d' % (J, J_), ValueWarning)
C:\Users\pc\Anaconda3\lib\site-packages\statsmodels\base\model.py:1532: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 5, but rank is 2
  'rank is %d' % (J, J_), ValueWarning)
C:\Users\pc\Anaconda3\lib\site-packages\statsmodels\base\model.py:1532: ValueWarning: covariance of constrai

,d_5050_x,w_5050_x,w_5020_x,w_5060_x,w_5045_x,RS_x,RSF_x
d_5050_y,1.0000,0.0000,0.0000,0.0,0.0,0.0,0.0
w_5050_y,0.0000,1.0000,0.0000,0.0,0.0,0.0,0.0
w_5020_y,0.0000,0.0000,1.0000,0.0,0.0,0.0,0.0
w_5060_y,0.0000,0.0000,0.0000,1.0,0.0,0.0,0.0
w_5045_y,0.0000,0.0000,0.0000,0.0,1.0,0.0,0.0
RS_y,0.0004,0.0006,0.0005,0.0,0.0,1.0,0.0
RSF_y,0.0000,0.0000,0.0000,0.0,0.0,0.0,1.0


In [75]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen

def cointegration_test(df, alpha=0.05): 
    """Perform Johanson's Cointegration Test and Report Summary"""
    out = coint_johansen(df,-1,5)
    d = {'0.90':0, '0.95':1, '0.99':2}
    traces = out.lr1
    cvts = out.cvt[:, d[str(1-alpha)]]
    def adjust(val, length= 6): return str(val).ljust(length)

    # Summary
    print('Name   ::  Test Stat > C(95%)    =>   Signif  \n', '--'*20)
    for col, trace, cvt in zip(df.columns, traces, cvts):
        print(adjust(col), ':: ', adjust(round(trace,2), 9), ">", adjust(cvt, 8), ' =>  ' , trace > cvt)
        
cointegration_test(first)

Name   ::  Test Stat > C(95%)    =>   Signif  
 ----------------------------------------
d_5050 ::  5442.29   > 111.7797  =>   True
w_5050 ::  3408.31   > 83.9383   =>   True
w_5020 ::  1921.92   > 60.0627   =>   True
w_5060 ::  1047.6    > 40.1749   =>   True
w_5045 ::  396.34    > 24.2761   =>   True
RS     ::  27.61     > 12.3212   =>   True
RSF    ::  4.2       > 4.1296    =>   True


In [76]:
def adfuller_test(series, signif=0.05, name='', verbose=False):
    """Perform ADFuller to test for Stationarity of given series and print report"""
    r = adfuller(series, autolag='AIC')
    output = {'test_statistic':round(r[0], 4), 'pvalue':round(r[1], 4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
    p_value = output['pvalue'] 
    def adjust(val, length= 6): return str(val).ljust(length)

    # Print Summary
    print(f'    Augmented Dickey-Fuller Test on "{name}"', "\n   ", '-'*47)
    print(f' Null Hypothesis: Data has unit root. Non-Stationary.')
    print(f' Significance Level    = {signif}')
    print(f' Test Statistic        = {output["test_statistic"]}')
    print(f' No. Lags Chosen       = {output["n_lags"]}')

    for key,val in r[4].items():
        print(f' Critical value {adjust(key)} = {round(val, 3)}')

    if p_value <= signif:
        print(f" => P-Value = {p_value}. Rejecting Null Hypothesis.")
        print(f" => Series is Stationary.")
    else:
        print(f" => P-Value = {p_value}. Weak evidence to reject the Null Hypothesis.")
        print(f" => Series is Non-Stationary.")  

# ADF Test on each column
for name, column in first.iteritems():
    adfuller_test(column, name=column.name)
    print('\n')

    Augmented Dickey-Fuller Test on "d_5050" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -14.2691
 No. Lags Chosen       = 19
 Critical value 1%     = -3.431
 Critical value 5%     = -2.862
 Critical value 10%    = -2.567
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Series is Stationary.


    Augmented Dickey-Fuller Test on "w_5050" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -13.0978
 No. Lags Chosen       = 18
 Critical value 1%     = -3.431
 Critical value 5%     = -2.862
 Critical value 10%    = -2.567
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Series is Stationary.


    Augmented Dickey-Fuller Test on "w_5020" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level 

In [3]:
def invert_transformation(df_train, df_forecast, second_diff=False):
    """Revert back the differencing to get the forecast to original scale."""
    df_fc = df_forecast.copy()
    columns = df_train.columns
    for col in columns:        
        # Roll back 2nd Diff
        if second_diff:
            df_fc[str(col)+'_1d'] = (df_train[col].iloc[-1]-df_train[col].iloc[-2]) + df_fc[str(col)+'_2d'].cumsum()
        # Roll back 1st Diff
        df_fc[str(col)+'_forecast'] = df_train[col].iloc[-1] + df_fc[str(col)+'_1d'].cumsum()
    return df_fc

### iterate

In [77]:
for i in range(39,len(pred_date),3):
    date = pred_date.date[i]
#     print(date)
#     print(date-pd.DateOffset(years=4))
    first = df_5050[(df_5050.index < date)&(df_5050.index > (date-pd.DateOffset(years=4)))]
#     print(first.tail())
#     build model
    model = VAR(endog=first.values)
#     choose p
    x = model.select_order(maxlags=12)
    aic = x.aic
#     fit model
    model_fitted = model.fit(9)
#     forecast
    fc = model_fitted.forecast(model_fitted.y, steps=3)
    df_forecast = pd.DataFrame(fc, index=[first.index[len(first)-1]+pd.DateOffset(1),first.index[len(first)-1]+pd.DateOffset(2),first.index[len(first)-1]+pd.DateOffset(3)], columns=first.columns )
    df_5050 = df_5050.append(df_forecast,ignore_index=False)
    df_5050.index = pd.to_datetime(df_5050.index)
    df_5050 = df_5050.sort_index()
#     print(df_forecast.index)

In [78]:
df_5050

,d_5050,w_5050,w_5020,w_5060,w_5045,RS,RSF
1965-01-01,1320.00,219.0,163.0,110.0,168.93,1.2,1.0
1965-01-02,1650.00,272.0,210.0,178.0,182.48,0.3,1.0
1965-01-03,1950.00,315.0,260.0,200.0,169.97,0.1,1.0
1965-01-04,1870.00,304.0,270.0,195.0,150.31,0.0,7.0
1965-01-05,1870.00,304.0,265.0,196.0,133.09,1.2,1.0
...,...,...,...,...,...,...,...
2012-12-27,5988.73,701.0,808.0,739.0,244.61,10.7,4.0
2012-12-28,5991.16,701.0,804.0,751.0,246.22,1.3,4.0
2012-12-29,6345.64,731.0,811.0,780.0,217.47,0.1,4.0
2012-12-30,6357.67,732.0,824.0,773.0,182.86,1.4,4.0


In [79]:
output = pd.DataFrame(columns=df_5050.columns)
for date in pred_date.date:
    if date in df_5050.index:
        output = output.append(df_5050.loc[date,:])
output = output.drop(['d_5050','w_5020','w_5045','RS','RSF'],axis=1)
output.to_csv("keyi_predict.csv")

## for 5060

In [80]:
# load river data
data = pd.read_csv("river_data.csv", index_col=False) 

df = data[data.station_no.isin([6335020,6335050,6335045,6335070,6335060])]

# df.index = pd.to_datetime(station_1.date)
# df = station_1.drop("date",axis=1)
df = df[df.water_level!=-999]

stations = [6335020,6335050,6335045,6335060]
df_5020 = df[df.station_no==stations[0]]
df_5020.index = pd.to_datetime(df_5020.date)
del df_5020['date']
df_5050 = df[df.station_no==stations[1]]
df_5050.index = pd.to_datetime(df_5050.date)
del df_5050['date']
df_5045 = df[df.station_no==stations[2]]
df_5045.index = pd.to_datetime(df_5045.date)
del df_5045['date']
df_5060 = df[df.station_no==stations[3]]
df_5060.index = pd.to_datetime(df_5060.date)
del df_5060['date']
# add other stations' water level as features
df_5050['w_5020'] = df_5020.water_level
df_5050['w_5060'] =  df_5060.water_level
df_5050['w_5045'] =  df_5045.water_level
# add other stations' discharge as features
# df_5050['d_5020'] = df_5020.discharge
df_5050['d_5060'] =  df_5060.discharge
del df_5050['discharge']
# df_5050['d_5045'] =  df_5045.discharge
df_5050['RS'] =weather.RS
df_5050['RSF'] = weather.RSF
df_5050['RSF'] =df_5050['RSF'].interpolate(method='nearest')
df_5050 = df_5050.fillna(weather['RS'].mean())

df_5050 = df_5050.drop("station_no",axis=1)
df_5050 = df_5050.rename(columns = { 'water_level':'w_5050'})
df_5050

C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stab

,w_5050,w_5020,w_5060,w_5045,d_5060,RS,RSF
date,,,,,,,
1965-01-01,219.0,163.0,110.0,168.93,1240.000,1.2,1.0
1965-01-02,272.0,210.0,178.0,182.48,1680.000,0.3,1.0
1965-01-03,315.0,260.0,200.0,169.97,1820.000,0.1,1.0
1965-01-04,304.0,270.0,195.0,150.31,1790.000,0.0,7.0
1965-01-05,304.0,265.0,196.0,133.09,1800.000,1.2,1.0
...,...,...,...,...,...,...,...
2012-12-27,701.0,808.0,739.0,244.61,5856.063,10.7,4.0
2012-12-28,701.0,804.0,751.0,246.22,5993.476,1.3,4.0
2012-12-29,731.0,811.0,780.0,217.47,6341.632,0.1,4.0


In [84]:
df_5060 = df_5050.copy()

In [85]:
# one_year_pre = pd.DataFrame(columns=first.columns)
for i in range(39,len(pred_date),3):
    date = pred_date.date[i]
#     print(date)
#     print(date-pd.DateOffset(years=4))
    first = df_5060[(df_5060.index < date)&(df_5060.index > (date-pd.DateOffset(years=4)))]
#     print(first.tail())
#     build model
    model = VAR(endog=first.values)
#     choose p
    x = model.select_order(maxlags=12)
    aic = x.aic
#     fit model
    model_fitted = model.fit(9)
#     forecast
    fc = model_fitted.forecast(model_fitted.y, steps=3)
    df_forecast = pd.DataFrame(fc, index=[first.index[len(first)-1]+pd.DateOffset(1),first.index[len(first)-1]+pd.DateOffset(2),first.index[len(first)-1]+pd.DateOffset(3)], columns=first.columns )
    df_5060 = df_5060.append(df_forecast,ignore_index=False)
    df_5060.index = pd.to_datetime(df_5060.index)
    df_5060 = df_5060.sort_index()
#     print(df_forecast.index)

In [86]:
df_5060

,w_5050,w_5020,w_5060,w_5045,d_5060,RS,RSF
1965-01-01,219.0,163.0,110.0,168.93,1240.000,1.2,1.0
1965-01-02,272.0,210.0,178.0,182.48,1680.000,0.3,1.0
1965-01-03,315.0,260.0,200.0,169.97,1820.000,0.1,1.0
1965-01-04,304.0,270.0,195.0,150.31,1790.000,0.0,7.0
1965-01-05,304.0,265.0,196.0,133.09,1800.000,1.2,1.0
...,...,...,...,...,...,...,...
2012-12-27,701.0,808.0,739.0,244.61,5856.063,10.7,4.0
2012-12-28,701.0,804.0,751.0,246.22,5993.476,1.3,4.0
2012-12-29,731.0,811.0,780.0,217.47,6341.632,0.1,4.0
2012-12-30,732.0,824.0,773.0,182.86,6253.428,1.4,4.0


In [87]:
output = pd.DataFrame(columns=df_5060.columns)
for date in pred_date.date:
    if date in df_5060.index:
        output = output.append(df_5060.loc[date,:])
output = output.drop(['d_5060','w_5020','w_5045','w_5050','RS','RSF'],axis=1)
output.to_csv("keyi_predict_5060.csv")